In [1]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.4.0\n")
    
!pip install -r requirements.txt  --upgrade --user

In [2]:
import kfp.dsl as dsl
import kfp
from kfp import components

kfserving_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/kfserving/component.yaml')

@dsl.pipeline(
  name='KFServing pipeline',
  description='A pipeline for KFServing.'
)
def kfservingPipeline(
    action='apply',
    model_name='tensorflow-sample',
    model_uri='gs://kfserving-samples/models/tensorflow/flowers',
    namespace='kubeflow-user-example-com',
    framework='tensorflow'):

    kfserving = kfserving_op(action = action,
                             model_name=model_name,
                             model_uri=model_uri,
                             namespace=namespace,
                             framework=framework).set_image_pull_policy('Always')



In [3]:
# Compile pipeline
kfp.compiler.Compiler().compile(kfservingPipeline, 'tf-flower.zip')

In [17]:
# note: please disable sidecar-injection on kubeflow-user namespace,
# as knative-serving / knative-events are not enabled istio sidecar.
# failed to disable could get RBAC error


!(echo -n '{"instances":[{"image_bytes": {"b64": "'; base64 2.jpg; echo '"}, "key": "1"}]}') | \
curl -XPOST -vvv -H "Host: tensorflow-sample.kubeflow-user-example-com.example.com" \
-H "Cookie: authservice_session=MTYxODc5OTE4OHxOd3dBTkVsWVIwSlFUek5KTTBRMVZrMVBWMHBGTjFkS1NsbFlSa2xSV0RST1ZEUTBVRkJaTXpWRVYwVk5UVUUyU1RZMk5rdEZWVUU9fAthoGZVLBIJdUEMixPohH4DTWNNIEEs5lzczYSfYicv" \
-d @- \
http://istio-ingressgateway.istio-system/v1/models/tensorflow-sample:predict

        
        

Note: Unnecessary use of -X or --request, POST is already inferred.
*   Trying 10.103.239.42:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system (10.103.239.42) port 80 (#0)
> POST /v1/models/tensorflow-sample:predict HTTP/1.1
> Host: tensorflow-sample.kubeflow-user.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYxODc5OTE4OHxOd3dBTkVsWVIwSlFUek5KTTBRMVZrMVBWMHBGTjFkS1NsbFlSa2xSV0RST1ZEUTBVRkJaTXpWRVYwVk5UVUUyU1RZMk5rdEZWVUU9fAthoGZVLBIJdUEMixPohH4DTWNNIEEs5lzczYSfYicv
> Content-Length: 97252
> Content-Type: application/x-www-form-urlencoded
> Expect: 100-continue
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 100 Continue
* We are completely uploaded and fine
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 218
< content-type: application/json
< date: Mon, 19 Apr 2021 09:12:09 GMT
< x-envoy-upstream-service-time: 644
< server: envoy
< 
{
    "predictions": [
        {
            "scores": [2.